In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
font_path = './NanumFontSetup_TTF_BARUNGOTHIC/NanumBarunGothic.ttf'
import matplotlib.font_manager as fm
fontprop = fm.FontProperties(fname=font_path, size=18)
import seaborn as sns
import shap

In [2]:
%%time
# Read Data
basedir = './Training/training_data_ver_16/'
training_data_df = pd.read_csv(basedir + 'training_data_ver_16.csv').drop(columns=['주택가격지수']).dropna()
#training_data_df = training_data_df[(training_data_df['가격면적'] >= 200) & (training_data_df['가격면적'] <= 1500)]
training_data_df.columns = [col.replace(' ','_').replace('·','_') for col in training_data_df.columns]

Wall time: 1min 26s


In [3]:
training_data_df = training_data_df[(training_data_df['가격면적'] >= 200) & (training_data_df['가격면적'] <= 3000)]

In [4]:
training_data_df['호수대비부속건축물면적'] = training_data_df['부속건축물면적'] / training_data_df['총호수']

In [5]:
training_data_df = training_data_df[(training_data_df['호수대비부속건축물면적'] < 100)]

In [6]:
training_data_df = training_data_df.reset_index(drop=True)

In [7]:
cols_to_log = ['총건축면적', '총연면적', '총용적률연면적', '토지면적', '부속건축물면적']
for col in cols_to_log:
    training_data_df[col] = training_data_df[col].apply(np.log1p)

In [8]:
cat_cols_df = pd.read_csv(basedir + 'cat_cols.csv')

In [9]:
cat_cols_list = [col.replace(' ','_').replace('·','_') for col in cat_cols_df['colname'].tolist()]
len(cat_cols_list)

296

In [10]:
cat_cols = [col for col in training_data_df.columns if col in cat_cols_list]
len(cat_cols)

296

In [11]:
cat_cols[:10]

['건물나이',
 'trade_type',
 '표준지여부',
 '지목명',
 '용도지역명1',
 '용도지역명2',
 '토지이동상황',
 '지형높이',
 '지형형상',
 '도로접면']

In [12]:
training_data_df['토지면적over총연면적times공시지가'] = training_data_df['토지면적'] / training_data_df['총연면적'] * training_data_df['공시지가(만원)']

In [13]:
shap_summary_df = pd.read_csv(basedir + 'shap_top_36.csv', index_col=[0]).sort_values(['shap_summary_mean'], ascending=False)
print(shap_summary_df.shape)
shap_summary_df.head()

(36, 6)


,model_0_shap_summary,model_1_shap_summary,model_2_shap_summary,model_3_shap_summary,model_4_shap_summary,shap_summary_mean
trade_type,93.883975,96.789440,95.676615,91.307413,95.812986,94.694086
토지면적over총연면적times공시지가,71.798361,75.450018,74.559728,75.656161,74.839162,74.460686
year_linear,66.046958,67.115133,64.616693,62.829061,66.577451,65.437059
건축년도,50.745131,50.318326,49.530523,51.125563,49.264659,50.196840
전용면적,37.663049,40.305701,35.925046,37.538836,37.998428,37.886212


In [14]:
shap_summary_df.tail(50)

,model_0_shap_summary,model_1_shap_summary,model_2_shap_summary,model_3_shap_summary,model_4_shap_summary,shap_summary_mean
trade_type,93.883975,96.789440,95.676615,91.307413,95.812986,94.694086
토지면적over총연면적times공시지가,71.798361,75.450018,74.559728,75.656161,74.839162,74.460686
year_linear,66.046958,67.115133,64.616693,62.829061,66.577451,65.437059
건축년도,50.745131,50.318326,49.530523,51.125563,49.264659,50.196840
전용면적,37.663049,40.305701,35.925046,37.538836,37.998428,37.886212
ynorm,32.979992,34.373101,33.867562,34.391293,32.689793,33.660348
xy,23.269583,22.479291,20.672874,22.702364,24.383246,22.701472
x_2nd,20.439473,20.525790,17.587809,19.614992,19.215703,19.476753
층mean,13.221159,12.170351,11.587628,13.440068,13.378779,12.759597
주건축물수,12.342852,11.590904,12.635774,10.983418,11.691039,11.848797


In [15]:
target_cols = ['금액(만원)', '가격면적', 'target_log_transformed', 'targetarea_log_transformed', '지번주소']

In [16]:
from sklearn.model_selection import KFold, GroupKFold
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

In [17]:
shap_summary1_df = shap_summary_df.copy()

In [18]:
for i in range(20):
    gc.collect()
    shortened_df = shap_summary1_df.iloc[:35-i,:]
    training_data_df = training_data_df[shortened_df.index.tolist()+target_cols]
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    cat_cols = [col for col in cat_cols if col in training_data_df.columns.tolist()]
    
    print(shortened_df.shape)

    oof = np.zeros(training_data_df.shape[0])

    for train_idx, val_idx in kf.split(training_data_df):
        train_data = training_data_df.loc[train_idx]
        val_data = training_data_df.loc[val_idx]

        x_train = train_data.drop(columns=target_cols)
        y_train = train_data['가격면적']

        x_val = val_data.drop(columns=target_cols)
        y_val = val_data['가격면적']

        params = {
            'objective': 'huber',
            'learning_rate': 10,
            'seed': 42,
            'max_depth': 12,
            'num_leaves': 37,
            'lambda_l2': 0,
            'metric': 'huber',
            'num_threads': 6,
        }

        train_dataset = lgb.Dataset(x_train, label=y_train, categorical_feature=cat_cols)
        val_dataset = lgb.Dataset(x_val, label=y_val, categorical_feature=cat_cols)

        num_rounds = 10000

        bst = lgb.train(params, train_dataset, num_rounds, valid_sets=[val_dataset], early_stopping_rounds=500, verbose_eval=1000)

        preds = bst.predict(x_val, num_iteration=bst.best_iteration)
        oof[val_idx] = preds
        
    print(np.sqrt(mean_squared_error(training_data_df['가격면적'], oof)))

(35, 6)


Using categorical_feature in Dataset.


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.130848 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4882
[LightGBM] [Info] Number of data points in the train set: 2926580, number of used features: 35


Overriding the parameters from Reference Dataset.
categorical_column in param dict is overridden.


[LightGBM] [Info] Start training from score 524.640869
Training until validation scores don't improve for 500 rounds
[1000]	valid_0's huber: 41.7745
[2000]	valid_0's huber: 38.1895
[3000]	valid_0's huber: 36.6206
[4000]	valid_0's huber: 35.4938
[5000]	valid_0's huber: 35.0119
[6000]	valid_0's huber: 34.7615
[7000]	valid_0's huber: 34.6447
[8000]	valid_0's huber: 34.5933
[9000]	valid_0's huber: 34.5085
[10000]	valid_0's huber: 34.4539
Did not meet early stopping. Best iteration is:
[9959]	valid_0's huber: 34.4536
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.138329 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4883
[LightGBM] [Info] Number of data points in the train set: 2926581, number of used features: 35
[LightGBM] [Info] Start training from score 524.755320
Training until validation scores don't improve for 500 rounds
[1000]	

...

[LightGBM] [Info] Start training from score 524.640869
Training until validation scores don't improve for 500 rounds
[1000]	valid_0's huber: 43.3784
[2000]	valid_0's huber: 39.6952
[3000]	valid_0's huber: 38.054
[4000]	valid_0's huber: 37.1183
[5000]	valid_0's huber: 36.5924
[6000]	valid_0's huber: 36.2613
[7000]	valid_0's huber: 36.0693
[8000]	valid_0's huber: 35.9411
[9000]	valid_0's huber: 35.8725
[10000]	valid_0's huber: 35.803
Did not meet early stopping. Best iteration is:
[9993]	valid_0's huber: 35.8026
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.069396 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3116
[LightGBM] [Info] Number of data points in the train set: 2926581, number of used features: 17
[LightGBM] [Info] Start training from score 524.755320
Training until validation scores don't improve for 500 rounds
[1000]	va

Using categorical_feature in Dataset.


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034057 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2863
[LightGBM] [Info] Number of data points in the train set: 2926580, number of used features: 16


Overriding the parameters from Reference Dataset.
categorical_column in param dict is overridden.


[LightGBM] [Info] Start training from score 524.640869
Training until validation scores don't improve for 500 rounds
[1000]	valid_0's huber: 43.3284
[2000]	valid_0's huber: 39.7978
[3000]	valid_0's huber: 38.0565
[4000]	valid_0's huber: 37.1097
[5000]	valid_0's huber: 36.5397
[6000]	valid_0's huber: 36.1968
[7000]	valid_0's huber: 36.0439
[8000]	valid_0's huber: 35.9132
[9000]	valid_0's huber: 35.8627
[10000]	valid_0's huber: 35.7938
Did not meet early stopping. Best iteration is:
[9994]	valid_0's huber: 35.7934
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034675 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2861
[LightGBM] [Info] Number of data points in the train set: 2926581, number of used features: 16
[LightGBM] [Info] Start training from score 524.755320
Training until validation scores don't improve for 500 rounds
[1000]	